# U-Net for Road Segmentation

## Required Folder Structure

The working folder contains the following subfolders:
* `training` Contains:
    * `groundtruth` Contains the groudtruth images (format: `satImage_xxx.png`)
    * `images` Contains the images to segment (format: `satImage_xxx.png`)
* `predictions_submission` Will contain the images predicted to create a submission
* `test_set_images` Contains the test images used to create a submission (format: `test_x.png` or `test_xx.png`)

This folder also contains 2 files:
* `road_segmentation_model.h5` Saved version the model. Allows to save the weights of the previous training that was done
* `smooth_tiled_prediction.py` Present in the GitHub repo.

In another folder we will store the autogenerated images. This folder should be called `augmented_set`.
In this folder there must be 2 subfolders:
* `groundtruth` contains autogenerated groundtruths
* `images` contains autogenerates images

## Pipeline

### Imports

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import argparse
import datetime
import os
import random
import re
import sys

from itertools import chain
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from PIL import Image
from skimage.io import imread, imshow
from skimage.transform import resize
import sklearn.metrics
from smooth_tiled_predictions import predict_img_with_smooth_windowing
from time import strftime
from tqdm import tqdm

### Constant Definitions

#### Non-Tweakable

In [ ]:
# Image definition
IMG_WIDTH    = 400
IMG_HEIGHT   = 400
IMG_CHANNELS = 3
PIXEL_DEPTH  = 255

# Folder definitions
IMAGE_DATA_PATH           = 'training/images/'
MASK_DATA_PATH            = 'training/groundtruth/'
MODEL_SAVE_LOCATION       = 'road_segmentation_model.h5'
SUBMISSION_DATA_DIR       = 'test_set_images/'
PREDICTION_SUBMISSION_DIR = 'predictions_submission/'

IMAGES_FILENAMES = os.listdir(IMAGE_DATA_PATH)

# Image generation
OUTPUT_DATA_IMAGE_PATH = 'augmented_set/'

# Seeding
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed = SEED
tf.random.set_seed(SEED)

# Checkpoints
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir  = os.path.dirname(checkpoint_path)

#### Tweakable

In [ ]:
# Image generation
GENERATE_NEW_IMG   = False
USE_GENERATED_IMG  = True
IMG_TO_GEN_PER_IMG = 100

# Load existing model
USE_SAVED_MODEL = True

# F1-score estimation
NUMBER_OF_IMG_TO_TEST = 10

# Predictions
RUN_PREDICTIONS_ON_TEST_IMAGES = True

### Image Generation

In [ ]:
if GENERATE_NEW_IMG:
    # load the input image, convert it to a NumPy array, and then
    # reshape it to have an extra dimension
    for img in tqdm(IMAGES_FILENAMES):
        image = load_img(IMAGE_DATA_PATH+img)
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)

        truth = load_img(MASK_DATA_PATH+img)
        truth = img_to_array(truth)
        truth = np.expand_dims(truth, axis=0)
        # construct the image generator for data augmentation then
        # initialize the total number of images generated thus far
        aug = ImageDataGenerator(rotation_range=360,
                  zoom_range=0.3,
                  brightness_range=[0.7,1],
                  width_shift_range=0.1,
                  height_shift_range=0.1,
                  vertical_flip=True,
                  shear_range=0.15,
                  horizontal_flip=True,
                  fill_mode="reflect")
        total = 0

        # construct the actual Python generator
        imageGen = aug.flow(image, y=truth, batch_size=1, save_to_dir=OUTPUT_DATA_IMAGE_PATH + "images",
          save_prefix=img.split(".")[0], save_format="png", seed = SEED )
        truthGen = aug.flow(truth, y=truth, batch_size=1, save_to_dir=OUTPUT_DATA_IMAGE_PATH + "groundtruth",
          save_prefix=img.split(".")[0], save_format="png", seed = SEED )
        # loop over examples from our image data augmentation generator
        for image in imageGen:
            # increment our counter
            total += 1

            # if we have reached the specified number of examples, break
            # from the loop
            if total == IMG_TO_GEN_PER_IMG:
                break

        total = 0
        for image in truthGen:
            # increment our counter
            total += 1

            # if we have reached the specified number of examples, break
            # from the loop
            if total == IMG_TO_GEN_PER_IMG:
                break

if(USE_GENERATED_IMG):
    print("[INFO]: Updating images_filename")
    IMAGE_DATA_PATH = OUTPUT_DATA_IMAGE_PATH+'images/'
    MASK_DATA_PATH = OUTPUT_DATA_IMAGE_PATH+ 'groundtruth/'
    print("[INFO]: new MASK_DATA_PATH : "+ MASK_DATA_PATH)
    print("[INFO]: new IMAGE_DATA_PATH : " + IMAGE_DATA_PATH)
    IMAGES_FILENAMES = os.listdir(IMAGE_DATA_PATH)
    print("[INFO]: There are " + str(len(IMAGES_FILENAMES)) + " found")

### Image Loading

In [ ]:
np.random.seed = SEED
print("[INFO]: Loading images into RAM", flush = True)
X = np.zeros((len(IMAGES_FILENAMES), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y = np.zeros((len(IMAGES_FILENAMES), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

for n, filename in tqdm(enumerate(IMAGES_FILENAMES), total=len(IMAGES_FILENAMES)):   
    img = imread(IMAGE_DATA_PATH + filename)[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X[n] = img
    mask = imread(MASK_DATA_PATH + filename)
    mask = np.expand_dims(resize(mask, (IMG_HEIGHT, IMG_WIDTH), mode='constant',
                                      preserve_range=True), axis=-1)
    if USE_GENERATED_IMG:
        Y[n] = mask[:,:,0]
    else:
        Y[n] = mask

x_train=X 
y_train=Y

### Model Building

In [ ]:
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH,+ IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
 
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
 
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(c5)
 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                            padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

### Saved Model Loading

In [ ]:
if USE_SAVED_MODEL:
    if not os.path.isfile(MODEL_SAVE_LOCATION):
        print("[ERROR]: Could not locate file for model weights. Proceding without loading weights.")
    else:
        model.load_weights(MODEL_SAVE_LOCATION)
        print("[INFO]: Loading saved model weights")

### Checkpoints for Training

In [ ]:
%load_ext tensorboard
#!rm -rf ./logs/ 

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)
callbacks = [
  #tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  tensorboard_callback,
  cp_callback 
]
%tensorboard --logdir logs/fit

### Training

In [ ]:
%%time

for i in range(10):
    model.save_weights(f'road_segmentation_model_{i}.h5', overwrite=True)
    results = model.fit(X, Y, validation_split=0.1, batch_size=32, epochs=100, callbacks=callbacks, shuffle=True)

model.save_weights(MODEL_SAVE_LOCATION, overwrite=True)

### F1-Score Estimation

In [ ]:
 #https://github.com/kiteco/kite-python-blog-post-code/blob/master/image-segmentation/image_segmentation.py

"""
Helper functions used to estimate F1 score.
"""

def get_f1_score(groundtruth_list, predicted_list):
    """Return f1 score covering edge cases"""

    tn, fp, fn, tp = get_confusion_matrix_elements(groundtruth_list, predicted_list)
    
    if _all_class_0_predicted_as_class_0(groundtruth_list, predicted_list) is True:
        f1_score = 1
    elif _all_class_1_predicted_as_class_1(groundtruth_list, predicted_list) is True:
        f1_score = 1
    else:
        f1_score = (2 * tp) / ((2 * tp) + fp + fn)

    return f1_score

def get_confusion_matrix_elements(groundtruth_list, predicted_list):
    """returns confusion matrix elements i.e TN, FP, FN, TP as floats
	See example code for helper function definitions
    """
    _assert_valid_lists(groundtruth_list, predicted_list)

    if _all_class_1_predicted_as_class_1(groundtruth_list, predicted_list) is True:
        tn, fp, fn, tp = 0, 0, 0, np.float64(len(groundtruth_list))

    elif _all_class_0_predicted_as_class_0(groundtruth_list, predicted_list) is True:
        tn, fp, fn, tp = np.float64(len(groundtruth_list)), 0, 0, 0

    else:
        tn, fp, fn, tp = sklearn.metrics.confusion_matrix(groundtruth_list, predicted_list).ravel()
        tn, fp, fn, tp = np.float64(tn), np.float64(fp), np.float64(fn), np.float64(tp)

    return tn, fp, fn, tp

def _assert_valid_lists(groundtruth_list, predicted_list):
    assert len(groundtruth_list) == len(predicted_list)
    for unique_element in np.unique(groundtruth_list).tolist():
        assert unique_element in [0, 1]

def _all_class_1_predicted_as_class_1(groundtruth_list, predicted_list):
    _assert_valid_lists(groundtruth_list, predicted_list)
    return np.unique(groundtruth_list).tolist() == np.unique(predicted_list).tolist() == [1]


def _all_class_0_predicted_as_class_0(groundtruth_list, predicted_list):
    _assert_valid_lists(groundtruth_list, predicted_list)
    return np.unique(groundtruth_list).tolist() == np.unique(predicted_list).tolist() == [0]

print("[INFO]: Estimating F1 score", flush=True)
step = 0.001
start = 0.51
stop = 0.53
F1s = []
thresholds = np.arange(start,stop+step,step)
for threshold in thresholds:
    F1_accumulator = 0

    for idx in tqdm(range(NUMBER_OF_IMG_TO_TEST)):
        x=np.array(x_train[idx])
        x=np.expand_dims(x, axis=0)
        predict = model.predict(x, verbose=0)

        predict = (predict > threshold).astype(np.uint8)
        f1 = get_f1_score(y_train[idx].flatten(),predict[0].flatten())
        F1_accumulator += f1
    #print("\nAverage F1: " + str(F1_accumulator / NUMBER_OF_IMG_TO_TEST))
    F1s.append(F1_accumulator / NUMBER_OF_IMG_TO_TEST)
best_index = np.argmax(F1s)
best_threshold = thresholds[best_index]
best_F1 = F1s[best_index]
print(f"\nThe best threshold is {best_threshold}, yielding an average F1 score of {best_F1*100:.2f}")
plt.plot(thresholds, F1s);

### Image Demo on Training Set

In [ ]:
def get_prediction(img):
    x=np.array(img)
    x=np.expand_dims(x, axis=0)
    predict = model.predict(x, verbose=1)
    print(predict.shape)
    predict = (predict > best_threshold).astype(np.uint8)
    predict = np.squeeze(predict[0])
    print(predict.shape)

    return predict

idx = 4
prediction = get_prediction(x_train[idx])

print("- Original Image:")
imshow(x_train[idx])
plt.show()

print("- Prediction:")
imshow(prediction)
plt.show()

print("- Original Groundtruth:")
imshow(np.squeeze(y_train[idx]));

### Running Predictions on Test Set

In [ ]:
def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * PIXEL_DEPTH).round().astype(np.uint8)
    return rimg

predictions = []
if RUN_PREDICTIONS_ON_TEST_IMAGES:
    print("[INFO]: Running prediction on submission set")
    if not os.path.isdir(PREDICTION_SUBMISSION_DIR):
        os.mkdir(PREDICTION_SUBMISSION_DIR)
    for i in range(1, 51):
        pimg = imread(SUBMISSION_DATA_DIR + f"test_{i}.png")[:,:,:IMG_CHANNELS]
        predictions.append(predict_img_with_smooth_windowing(
            pimg,
            window_size=IMG_WIDTH,
            subdivisions=2,  # Minimal amount of overlap for windowing. Must be an even number.
            nb_classes=1,
                pred_func=(
                    lambda img_batch_subdiv: model.predict(img_batch_subdiv)
                )
            )
        )
else:
    print("[INFO]: Skipping predicting test images")

### Prediction Saving

In [ ]:
if RUN_PREDICTIONS_ON_TEST_IMAGES:
    print("[INFO]: Writing prediction to drive")

    ROAD_THRESHOLD = 0.25#0.1

    pred = np.array(predictions.copy())
    for i in range(1, 51):
        pimg = pred[i-1]
        w = pimg.shape[0]
        h = pimg.shape[1]
        cimg = np.zeros((w, h, 3), dtype=np.uint8)
        pimg = (pimg > best_threshold).astype(np.uint8)
        pimg8 = np.squeeze(img_float_to_uint8(pimg))
        cimg[:, :, 0] = pimg8
        cimg[:, :, 1] = pimg8
        cimg[:, :, 2] = pimg8
        Image.fromarray(cimg).save(PREDICTION_SUBMISSION_DIR + f"gt_{i}.png")
else:
    print("[INFO]: Skipping write of predictions to disk")

### AICrowd Submission Creation

In [ ]:
# Creating ouput for submission

foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > foreground_threshold:
        return 1
    else:
        return 0

def mask_to_submission_strings(image_filename):
    """Reads a single image and outputs the strings that should go into the submission file"""
    img_number = int(re.search(r"\d+", image_filename).group(0))
    im = mpimg.imread(image_filename)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def masks_to_submission(submission_filename, *image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames[0:]:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn))


time = strftime("%Y%m%dT%H%M%S")
submission_filename = f'submission-{time}.csv'
image_filenames = []
for i in range(1, 51):
    image_filename = f'{PREDICTION_SUBMISSION_DIR}gt_{i}.png'
    image_filenames.append(image_filename)
masks_to_submission(submission_filename, *image_filenames)

## Hyperparameter Tweaking

### Prediction Threshold

In [ ]:
i = 23
input_img = imread(SUBMISSION_DATA_DIR + f"test_{i}.png")[:,:,:IMG_CHANNELS]

predictions_smooth = predict_img_with_smooth_windowing(
    input_img,
    window_size=IMG_WIDTH,
    subdivisions=2,  # Minimal amount of overlap for windowing. Must be an even number.
    nb_classes=1,
    pred_func=(
        lambda img_batch_subdiv: model.predict(img_batch_subdiv)
    )
)

predict = (predictions_smooth > 0.25).astype(np.uint8)

imshow(np.squeeze(predict))
plt.show()
imshow(input_img)

## References

- https://androidkt.com/tensorflow-keras-unet-for-image-image-segmentation/

- https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277

- https://arxiv.org/pdf/1505.04597.pdf